In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import *
from keras.utils.np_utils import to_categorical
from keras.initializers import Constant
import re
import os 
import nltk

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#List of strings to remove from the corpus
unique_type_list = ['INFJ', 
                    'ENTP', 
                    'INTP', 
                    'INTJ', 
                    'ENTJ', 
                    'ENFJ', 
                    'INFP', 
                    'ENFP',
                    'ISFP', 
                    'ISTP', 
                    'ISFJ', 
                    'ISTJ', 
                    'ESTP', 
                    'ESFP', 
                    'ESTJ', 
                    'ESFJ']

unique_type_list = [x.lower() for x in unique_type_list]

# Initialize for Lemmatization
# stemmer = PorterStemmer()
# lemmatiser = WordNetLemmatizer()

def pre_process_data(data, remove_stop_words = False, remove_mbti_profiles = True):
    list_personality = []
    list_posts = []
    list_features = []
    len_data = len(data)
    i=0
    
    for row in data.iterrows():
        i+=1
        if (i % 500 == 0 or i == 1 or i == len_data):
            print("%s of %s rows" % (i, len_data))

        ##### Remove and clean comments
        posts = row[1].posts
        temp = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', posts)
        temp = re.sub("[^a-zA-Z]", " ", temp)
        temp = re.sub(' +', ' ', temp).lower()
        if remove_stop_words:
            temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ') if w not in stopwords.words("english")])
#         else:
#             temp = " ".join([lemmatiser.lemmatize(w) for w in temp.split(' ')])
            
        if remove_mbti_profiles:
            for t in unique_type_list:
                temp = temp.replace(t,"")

        word_count = len(temp.split(' '))
        
        #Filter for minimum 50 words and max 2000 words
        if(word_count>50 and word_count<2000):
            type_labelized = translate_personality(row[1].type)
            list_personality.append(type_labelized)
            list_posts.append(temp)
            #Extract post-level features
            list_features = extract_post_level_features(temp)

    list_posts = np.array(list_posts)
    list_personality = np.array(list_personality)
    return list_posts, list_personality

def extract_post_level_features(post):
    word_count = len(post.split(' '))
    
    #Extract POS-tags
    tokens = nltk.word_tokenize(post)
    pos_tags = nltk.pos_tag(tokens)
    pos_tags = [x[1] for x in pos_tags]
    noun_counts = pos_tags.count('NN')
    print((post), noun_counts)